In [2]:
print("X_train.shape:", X_train.shape)
print("X_test.shape: ", X_test.shape)


X_train.shape: (3689, 5)
X_test.shape:  (0, 5)


In [3]:
print("Mínima fecha:", df_model['fecha'].min())
print("Máxima fecha:", df_model['fecha'].max())

Mínima fecha: 2021-09-13 10:41:41
Máxima fecha: 2024-02-02 17:37:32


In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# 0) Carga de datos
df = pd.read_csv("dataset_combinado.csv", parse_dates=['fecha'])

# 1) Filtrar sólo datos reales
df_real = df[df['usuario'].isin(['usuario_strong', 'usuario_weightlifting'])].copy()

# 2) Ordenar y crear la variable objetivo: próxima carga
df_real = df_real.sort_values(['usuario', 'ejercicio', 'fecha'])
df_real['peso_next'] = df_real.groupby(['usuario','ejercicio'])['peso (kg)'].shift(-1)

# 3) Crear features de lag y time-deltas
df_real['peso_prev'] = df_real.groupby(['usuario','ejercicio'])['peso (kg)'].shift(1)
df_real['reps_prev'] = df_real.groupby(['usuario','ejercicio'])['reps'].shift(1)
df_real['rir_prev']  = df_real.groupby(['usuario','ejercicio'])['rir'].shift(1)
df_real['delta_peso'] = df_real['peso (kg)'] - df_real['peso_prev']
df_real['dias_entre_sesiones'] = df_real.groupby(['usuario','ejercicio'])['fecha'].diff().dt.days

# 4) Eliminar filas con NaN en features u objetivo
features = ['peso_prev','reps_prev','rir_prev','delta_peso','dias_entre_sesiones']
df_model = df_real.dropna(subset=features + ['peso_next']).copy()

# 5) Split temporal: antes de 2024-01-01 → train; desde 2024-01-01 → test
split_date = '2024-01-03'
train = df_model[df_model['fecha'] < split_date]
test  = df_model[df_model['fecha'] >= split_date]

X_train, y_train = train[features], train['peso_next']
X_test,  y_test  = test[features],  test['peso_next']

print(f"Train shape: {X_train.shape}  Test shape: {X_test.shape}")

# 6) Entrenar modelo baseline
model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 7) Predecir y evaluar
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"MAE (solo datos reales): {mae:.2f} kg")

# 8) Ver primeras predicciones vs realidad
comparison = test[['usuario','ejercicio','fecha','peso (kg)','peso_next']].copy()
comparison['predicción'] = preds
print(comparison.head(5))


Train shape: (3503, 5)  Test shape: (186, 5)
MAE (solo datos reales): 1.45 kg
             usuario              ejercicio               fecha  peso (kg)  \
3821  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3822  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3823  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   
3824  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       25.0   
3837  usuario_strong  Bicep Curl (Dumbbell) 2024-01-04 11:45:06       30.0   

      peso_next  predicción  
3821       30.0   29.907807  
3822       30.0   29.907807  
3823       25.0   29.907807  
3824       30.0   29.907807  
3837       30.0   29.330357  


In [10]:
# Regla +2 % comparación
preds_rule = test['peso (kg)'] * 1.02
mae_rule = mean_absolute_error(y_test, preds_rule)
print(f"MAE regla +2 %: {mae_rule:.2f} kg")


MAE regla +2 %: 1.44 kg


In [1]:
import pandas as pd

# 1) Carga el dataset combinado
df = pd.read_csv("dataset_combinado.csv", parse_dates=['fecha'])

# 2) (Opcional) Filtrar sólo usuarios reales si quieres
usuarios_reales = ['usuario_strong', 'usuario_weightlifting']
df = df[df["usuario"].isin(usuarios_reales)].copy()

# 3) Asegúrate de que las columnas de fecha son datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# 4) Revisa rápidamente la estructura
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 13921 entries, 0 to 13920
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   usuario           13921 non-null  object        
 1   fecha             13921 non-null  datetime64[ns]
 2   ejercicio         13921 non-null  object        
 3   set               13921 non-null  int64         
 4   peso (kg)         13921 non-null  float64       
 5   reps              13921 non-null  int64         
 6   rpe               3767 non-null   float64       
 7   rir               3767 non-null   float64       
 8   cumplió_objetivo  13921 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 992.4+ KB
None
          usuario               fecha                ejercicio  set  \
0  usuario_strong 2021-09-13 10:41:41    Bench Press (Barbell)    1   
1  usuario_strong 2021-09-13 10:41:41    Bench Press (Barbell)  

In [3]:
# modelado_completo.py

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib

# 1) Carga y limpieza de datos
df = pd.read_csv("dataset_combinado.csv", parse_dates=['fecha'])
df = df.sort_values(['usuario', 'ejercicio', 'fecha', 'set'])

# 2) Crear variable objetivo: siguiente carga
df['peso_next'] = df.groupby(['usuario','ejercicio'])['peso (kg)'].shift(-1)

# 3) Ingeniería de features
df['peso_prev']  = df.groupby(['usuario','ejercicio'])['peso (kg)'].shift(1)
df['reps_prev']  = df.groupby(['usuario','ejercicio'])['reps'].shift(1)
df['rpe_prev']   = df.groupby(['usuario','ejercicio'])['rpe'].shift(1)
df['rir_prev']   = df.groupby(['usuario','ejercicio'])['rir'].shift(1)
df['delta_peso'] = df['peso (kg)'] - df['peso_prev']
df['dias_entre'] = df.groupby(['usuario','ejercicio'])['fecha'].diff().dt.days

# 4) Eliminar filas con valores nulos en features u objetivo
features = ['peso_prev','reps_prev','rpe_prev','rir_prev','delta_peso','dias_entre']
df_model = df.dropna(subset=features + ['peso_next']).copy()

# 5) División train/test (aleatorio 80/20)
X = df_model[features]
y = df_model['peso_next']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train shape: {X_train.shape}; Test shape: {X_test.shape}")

# 6) Entrenar modelo base (Decision Tree)
dt = DecisionTreeRegressor(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
preds_dt = dt.predict(X_test)
mae_dt = mean_absolute_error(y_test, preds_dt)
print(f"MAE Decision Tree: {mae_dt:.2f} kg")

# 7) Ajuste de hiperparámetros con GridSearchCV
param_grid = {
    'max_depth': [3,5,7,9],
    'min_samples_leaf': [1,5,10,20]
}
grid = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid, cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)
grid.fit(X_train, y_train)
best_dt = grid.best_estimator_
preds_gs = best_dt.predict(X_test)
mae_gs = mean_absolute_error(y_test, preds_gs)
print(f"Mejores parámetros DT: {grid.best_params_}")
print(f"MAE DT ajustado: {mae_gs:.2f} kg")

# 8) Probar RandomForest
rf = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
preds_rf = rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, preds_rf)
print(f"MAE RandomForest: {mae_rf:.2f} kg")

# 9) (Opcional) Probar XGBoost si está instalado
try:
    import xgboost as xgb
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100, max_depth=5,
        learning_rate=0.1, random_state=42,
        n_jobs=-1
    )
    xgb_reg.fit(X_train, y_train)
    preds_xgb = xgb_reg.predict(X_test)
    mae_xgb = mean_absolute_error(y_test, preds_xgb)
    print(f"MAE XGBoost: {mae_xgb:.2f} kg")
    final_model = xgb_reg if mae_xgb < min(mae_rf, mae_gs) else (rf if mae_rf < mae_gs else best_dt)
except ImportError:
    final_model = rf if mae_rf < mae_gs else best_dt

# 10) Guardar el modelo final
joblib.dump(final_model, "nextlift_model.pkl")
print("Modelo final guardado como nextlift_model.pkl")


Train shape: (5986, 6); Test shape: (1497, 6)
MAE Decision Tree: 2.23 kg
Mejores parámetros DT: {'max_depth': 9, 'min_samples_leaf': 1}
MAE DT ajustado: 0.97 kg
MAE RandomForest: 1.09 kg
MAE XGBoost: 1.00 kg
Modelo final guardado como nextlift_model.pkl
